In [16]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import BatchNormalization, InputSpec, add
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers, activations
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.utils import Sequence


In [17]:
import keras
import pandas as pd
import numpy as np

In [18]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
test_val = np.load('test_val.npy')
y_test = np.load('y_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [19]:
x_train.shape

(37065, 1011)

In [20]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

In [21]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [22]:
class DotProdSelfAttention(Layer):
    """The self-attention layer as in 'Attention is all you need'.
    paper reference: https://arxiv.org/abs/1706.03762
    
    """
    def __init__(self, units,
                 activation=None,
                 use_bias=False,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(DotProdSelfAttention, self).__init__(*kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(min_ndim=2)
        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) == 3
        input_dim = input_shape[-1]
        # We assume the output-dim of Q, K, V are the same
        self.kernels = dict.fromkeys(['Q', 'K', 'V'])
        for key, _ in self.kernels.items():
            self.kernels[key] = self.add_weight(shape=(input_dim, self.units),
                                                initializer=self.kernel_initializer,
                                                name='kernel_{}'.format(key),
                                                regularizer=self.kernel_regularizer,
                                                constraint=self.kernel_constraint)
        if self.use_bias:
            raise NotImplementedError
        super(DotProdSelfAttention, self).build(input_shape)
        
    def call(self, x):
        Q = K.dot(x, self.kernels['Q'])
        K_mat = K.dot(x, self.kernels['K'])
        V = K.dot(x, self.kernels['V'])
        attention = K.batch_dot(Q, K.permute_dimensions(K_mat, [0, 2, 1]))
        d_k = K.constant(self.units, dtype=K.floatx())
        attention = attention / K.sqrt(d_k)
        attention = K.batch_dot(K.softmax(attention, axis=-1), V)
        return attention
    
    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

In [23]:
class squash_function(Layer):
    def squash(s, axis=-1, epsilon=1e-7, name=None):
        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keep_dims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector

In [24]:
gru_len = 128
Routings = 5
Num_capsule = 100
Dim_capsule = 20
dropout_p = 0.3
rate_drop_dense = 0.3
filter_length = 128
num_classes = 20
max_words = 111396
maxlen = 1011
batch_size = 64

In [25]:
def encoder(input_tensor):
    

    """One encoder as in Attention Is All You Need"""

    # Sub-layer 1
    # Multi-Head Attention
    multiheads = []
    d_v = embed_size // n_heads
    for i in range(n_heads):
        multiheads.append(DotProdSelfAttention(d_v)(input_tensor))
    multiheads = concatenate(multiheads, axis=-1)
    multiheads = Conv1D(64, kernel_size=1, padding="valid", kernel_initializer="glorot_uniform")(multiheads)
    multiheads = Dense(embed_size)(multiheads)
    multiheads = Dropout(0.1)(multiheads)
    
    # Residual Connection
    res_con = add([input_tensor, multiheads])
    # Didn't use layer normalization, use Batch Normalization instead here
    res_con = squash_function()(res_con)
    
    # Sub-layer 2
    # 2 Feed forward layer
    ff1 = Dense(32, activation='relu')(res_con)
    ff2 = Dense(embed_size)(ff1)
    output = add([res_con, ff2])
    output = squash_function()(output)
    
    return output


In [26]:
import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [27]:
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))

In [30]:
filter_length = 128
num_classes = 20
max_words = 111396
maxlen = 1011
embed_size = 20
n_heads = 8
from sklearn.metrics import hamming_loss as hamming
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint


input1 = Input(shape=(maxlen,))
x = Embedding(max_words, 20, input_length=maxlen)(input1)

#x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
# Add positional encoding
#x = AddPositionalEncoding()(x)
#x = Dropout(0.1)(x)
for i in range(2):
    x = encoder(x)
# These are my own experiments
capsule = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=Routings,
                  share_weights=True)(x)
#x = Conv1D(64, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
attn = AttentionWeightedAverage()(x)
flat = Flatten()(x)
conc1 = concatenate([attn, flat])
conc2 = concatenate([max_pool,avg_pool])
conc = concatenate([conc1,conc2])
conc = squash_function()(conc)
conc = Dense(32, activation="relu")(conc)
conc = Dropout(0.1)(conc)

preds = Dense(num_classes, activation="sigmoid")(conc)
 
model = keras.Model(input1, preds)
 
model.summary()
 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Hamming_loss])

# import tensorflow as tf
# gpu_options = tf.GPUOptions(allow_growth=True)
# session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
# model.fit(x_train, y_train, batch_size=8, epochs=4)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1011)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1011, 20)     2227920     input_3[0][0]                    
__________________________________________________________________________________________________
dot_prod_self_attention_49 (Dot (None, 1011, 2)      120         embedding_3[0][0]                
__________________________________________________________________________________________________
dot_prod_self_attention_50 (Dot (None, 1011, 2)      120         embedding_3[0][0]                
__________________________________________________________________________________________________
dot_prod_s

In [31]:
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss

def _train_model(model, batch_size, train_x, train_y, test_val, y_val):
    num_labels = train_y.shape[1]
    patience = 5
    best_loss = -1
    best_weights = None
    best_epoch = 0
    
    current_epoch = 0
    
    while True:
        model.fit(x_train, y_train, batch_size=batch_size, epochs=1)
        y_pred = model.predict(test_val, batch_size=batch_size)

        total_loss = 0
        for j in range(num_labels):
            loss = log_loss(y_val[:, j], y_pred[:, j])
            total_loss += loss

        total_loss /= num_labels

        print("Epoch {0} loss {1} best_loss {2}".format(current_epoch, total_loss, best_loss))

        current_epoch += 1
        if total_loss < best_loss or best_loss == -1:
            best_loss = total_loss
            best_weights = model.get_weights()
            best_epoch = current_epoch
        else:
            if current_epoch - best_epoch == patience:
                break

    model.set_weights(best_weights)
    return model

In [32]:
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

batch_size = 8
history = _train_model(model,batch_size,x_train,y_train,test_val,y_val)

Epoch 1/1
37065/37065 [==============================] - 474s 13ms/step - loss: 0.4343 - Hamming_loss: 0.2130
Epoch 0 loss 0.41023484806351523 best_loss -1
Epoch 1/1
37065/37065 [==============================] - 471s 13ms/step - loss: 0.3946 - Hamming_loss: 0.1894
Epoch 1 loss 0.40425534166085625 best_loss 0.41023484806351523
Epoch 1/1
37065/37065 [==============================] - 472s 13ms/step - loss: 0.3660 - Hamming_loss: 0.1739
Epoch 2 loss 0.4079241859822423 best_loss 0.40425534166085625
Epoch 1/1
37065/37065 [==============================] - 472s 13ms/step - loss: 0.3428 - Hamming_loss: 0.1615
Epoch 3 loss nan best_loss 0.40425534166085625
Epoch 1/1
    8/37065 [..............................] - ETA: 8:17 - loss: 0.3210 - Hamming_loss: 0.1750

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


37065/37065 [==============================] - 471s 13ms/step - loss: 0.3249 - Hamming_loss: 0.1532
Epoch 4 loss nan best_loss 0.40425534166085625
Epoch 1/1
    8/37065 [..............................] - ETA: 8:08 - loss: 0.3535 - Hamming_loss: 0.1688

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

37065/37065 [==============================] - 471s 13ms/step - loss: 0.3117 - Hamming_loss: 0.1468
Epoch 5 loss nan best_loss 0.40425534166085625
Epoch 1/1
    8/37065 [..............................] - ETA: 8:00 - loss: 0.2710 - Hamming_loss: 0.1500

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

37065/37065 [==============================] - 471s 13ms/step - loss: 0.3008 - Hamming_loss: 0.1413


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

Epoch 6 loss nan best_loss 0.40425534166085625


In [33]:
y_pred_test = history.predict(x_test, batch_size=batch_size)
y_pred_val = history.predict(test_val,batch_size=batch_size)

In [34]:
y_pred_test = pd.DataFrame(y_pred_test)
y_pred_val = pd.DataFrame(y_pred_val)
y_pred_val.to_pickle("y_pred_transformer+capsule_val.pkl")
y_pred_test.to_pickle("y_pred_transformer+capsule_test.pkl")

In [35]:
print(y_pred[75],y_val[75])

NameError: name 'y_pred' is not defined

In [ ]:
y_pred_round = np.where(y_pred_test>0.5,1,0)

In [ ]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
print(jaccard_score(y_test,y_pred_round,average='micro'))
print(jaccard_score(y_test,y_pred_round,average='macro'))
print(roc_auc_score(y_test,y_pred_round))
print(hamming_loss(y_test,y_pred_round))

In [23]:
from keras.models import load_model
model.save('transformer+capsnet1.h5')

In [36]:
df = pd.read_pickle("./str+rad_text-combined.pkl")
df= df.drop(["SUBJECT_ID","text","HADM_ID"],axis=1)
x = df.iloc[:,:483]
y = df.iloc[:,483:]
from sklearn.model_selection import train_test_split
#train,test,val 
seed = 40
x_tra,x_test,y_tra,y_test = train_test_split(x,y,test_size=0.15,random_state=seed)
x_train,x_val,y_train,y_val = train_test_split(x_tra,y_tra,test_size = 0.2,random_state=seed)


%run helper_functions.ipynb

x_train_txt = np.load('x_train.npy') 
x_test_txt = np.load('x_test.npy')
test_val_txt = np.load('test_val.npy') 
y_test_txt = np.load('y_test.npy')
y_train_txt = np.load('y_train.npy')
y_val_txt = np.load('y_val.npy')
batch_size1 = 64

from keras.models import load_model
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True)
session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
conv1 = load_model('conv1.h5',custom_objects={'squash_function': squash_function,
                                              'AttentionWeightedAverage':AttentionWeightedAverage,
                                              'Hamming_loss':Hamming_loss
                                             })

conv3 = load_model('conv3.h5',custom_objects={'squash_function': squash_function,
                                              'AttentionWeightedAverage':AttentionWeightedAverage,
                                              'Hamming_loss':Hamming_loss
                                             })
conv_n_gru = load_model('conv_n_gru.h5',custom_objects={'squash_function': squash_function,
                                              'AttentionWeightedAverage':AttentionWeightedAverage,
                                              'Hamming_loss':Hamming_loss
                                             })
conv5 = load_model('conv5.h5',custom_objects={'squash_function': squash_function,
                                              'AttentionWeightedAverage':AttentionWeightedAverage,
                                              'Hamming_loss':Hamming_loss
                                             })
y_pred_conv3 = conv3.predict(test_val_txt, batch_size=batch_size1)
y_pred_conv5 = conv5.predict(test_val_txt, batch_size=batch_size1)
y_pred_conv1 = conv1.predict(test_val_txt, batch_size=batch_size1)
y_pred_conv_n_gru=conv_n_gru.predict(test_val_txt, batch_size=batch_size1)

#print(y_pred_conv3.shape,y_pred.shape,y_pred_lgb.shape)

# Structured models

from sklearn.externals import joblib
cat = joblib.load("/home/akshara/MIMIC-Project/Trained Models/CatBoostC.pkl")
lgb = joblib.load("/home/akshara/MIMIC-Project/try1/LGBM_40.pkl")

y_pred_cat = cat.predict(x_val)
y_pred_lgb = lgb.predict(x_val)

def corr(a, b):
    counts = 0
    for i in range(a.shape[0]):  
      # correlation
        print("Finding correlation row : {}".format(i))
      #print("Column to be measured: {}".format(prediction))
        cor = np.corrcoef(a[i,:],b[i,:])
        print("Pearson's correlation score: {}".format(cor))
        if(cor[0][1] < 0.8):  
            counts+=1
    return counts

y_pred_val = np.where(y_pred_val>0.5,1,0)
#l = [corr(y_pred_conv3, y_pred_conv1),corr(y_pred_conv3, y_pred_cat),corr(y_pred_conv1, y_pred_cat),corr(y_pred_conv3, y_pred_lgb),corr(y_pred_conv1, y_pred_lgb)] 
#l = [corr(y_pred_conv3, y_pred_lgb),corr(y_pred_conv1, y_pred_lgb),corr(y_pred_conv5, y_pred),corr(y_pred_conv_n_gru, y_pred_val)] 

#l

m = [corr(y_pred_conv3, y_pred_lgb),corr(y_pred_conv1, y_pred_lgb),corr(y_pred_conv5, y_pred_lgb),corr(y_pred_conv_n_gru, y_pred_lgb)
     ,corr(y_pred_lgb, y_pred_val),corr(y_pred_conv5, y_pred_val),
    corr(y_pred_conv3, y_pred_val),corr(y_pred_conv1, y_pred_val),corr(y_pred_conv_n_gru, y_pred_val),
    corr(y_pred_conv3, y_pred_conv1),corr(y_pred_conv3, y_pred_conv5),corr(y_pred_conv3, y_pred_conv_n_gru),
    ] 

m

y_pred1 = 0.14*y_pred_val + 0.14*y_pred_conv1 + 0.14*y_pred_conv3 + 0.14*y_pred_conv5 + 0.3*y_pred_lgb + 0.14*y_pred_conv_n_gru
y_pred_round = np.where(y_pred1>0.5,1,0)

#y_pred_round = np.where(y_pred>0.5,1,0)
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
print(jaccard_score(y_val,y_pred_round,average='micro'))
print(jaccard_score(y_val,y_pred_round,average='macro'))
print(roc_auc_score(y_val,y_pred_round))
print(hamming_loss(y_val,y_pred_round))

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Pearson's correlation score: [[1.         0.67399248]
 [0.67399248 1.        ]]
Finding correlation row : 5610
Pearson's correlation score: [[1.         0.62666623]
 [0.62666623 1.        ]]
Finding correlation row : 5611
Pearson's correlation score: [[1.         0.64592559]
 [0.64592559 1.        ]]
Finding correlation row : 5612
Pearson's correlation score: [[1.        0.7737656]
 [0.7737656 1.       ]]
Finding correlation row : 5613
Pearson's correlation score: [[1.         0.81498263]
 [0.81498263 1.        ]]
Finding correlation row : 5614
Pearson's correlation score: [[1.         0.69069738]
 [0.69069738 1.        ]]
Finding correlation row : 5615
Pearson's correlation score: [[1.        0.6027281]
 [0.6027281 1.       ]]
Finding correlation row : 5616
Pearson's correlation score: [[1.         0.74907902]
 [0.74907902 1.        ]]
Finding correlation row : 5617
Pearson's correlation score: [[1.        0.5362681]
 [0.5362681 1.       ]]
Finding correlation row : 5618
Pearson's cor

Pearson's correlation score: [[1.         0.73729959]
 [0.73729959 1.        ]]
Finding correlation row : 5989
Pearson's correlation score: [[1.         0.90524362]
 [0.90524362 1.        ]]
Finding correlation row : 5990
Pearson's correlation score: [[1.         0.77721646]
 [0.77721646 1.        ]]
Finding correlation row : 5991
Pearson's correlation score: [[1.         0.85256604]
 [0.85256604 1.        ]]
Finding correlation row : 5992
Pearson's correlation score: [[1.        0.7239429]
 [0.7239429 1.       ]]
Finding correlation row : 5993
Pearson's correlation score: [[1.         0.74046536]
 [0.74046536 1.        ]]
Finding correlation row : 5994
Pearson's correlation score: [[1.         0.59530501]
 [0.59530501 1.        ]]
Finding correlation row : 5995
Pearson's correlation score: [[1.        0.5143572]
 [0.5143572 1.       ]]
Finding correlation row : 5996
Pearson's correlation score: [[1.         0.82269158]
 [0.82269158 1.        ]]
Finding correlation row : 5997
Pearson's

Pearson's correlation score: [[1.         0.46481782]
 [0.46481782 1.        ]]
Finding correlation row : 6375
Pearson's correlation score: [[1.         0.86447983]
 [0.86447983 1.        ]]
Finding correlation row : 6376
Pearson's correlation score: [[1.         0.75335498]
 [0.75335498 1.        ]]
Finding correlation row : 6377
Pearson's correlation score: [[1.         0.49343136]
 [0.49343136 1.        ]]
Finding correlation row : 6378
Pearson's correlation score: [[1.        0.6978398]
 [0.6978398 1.       ]]
Finding correlation row : 6379
Pearson's correlation score: [[1.         0.74746932]
 [0.74746932 1.        ]]
Finding correlation row : 6380
Pearson's correlation score: [[1.         0.56724954]
 [0.56724954 1.        ]]
Finding correlation row : 6381
Pearson's correlation score: [[1.         0.66531798]
 [0.66531798 1.        ]]
Finding correlation row : 6382
Pearson's correlation score: [[1.         0.69531984]
 [0.69531984 1.        ]]
Finding correlation row : 6383
Pears

Pearson's correlation score: [[1.         0.52775275]
 [0.52775275 1.        ]]
Finding correlation row : 6791
Pearson's correlation score: [[1.         0.74521399]
 [0.74521399 1.        ]]
Finding correlation row : 6792
Pearson's correlation score: [[1.         0.41882219]
 [0.41882219 1.        ]]
Finding correlation row : 6793
Pearson's correlation score: [[1.         0.83734354]
 [0.83734354 1.        ]]
Finding correlation row : 6794
Pearson's correlation score: [[1.         0.83476691]
 [0.83476691 1.        ]]
Finding correlation row : 6795
Pearson's correlation score: [[1.         0.62622046]
 [0.62622046 1.        ]]
Finding correlation row : 6796
Pearson's correlation score: [[1.         0.46211284]
 [0.46211284 1.        ]]
Finding correlation row : 6797
Pearson's correlation score: [[1.         0.70124681]
 [0.70124681 1.        ]]
Finding correlation row : 6798
Pearson's correlation score: [[1.         0.57933945]
 [0.57933945 1.        ]]
Finding correlation row : 6799
P

...

Pearson's correlation score: [[1.         0.73356495]
 [0.73356495 1.        ]]
Finding correlation row : 2268
Pearson's correlation score: [[1.         0.68963322]
 [0.68963322 1.        ]]
Finding correlation row : 2269
Pearson's correlation score: [[1.         0.88380952]
 [0.88380952 1.        ]]
Finding correlation row : 2270
Pearson's correlation score: [[1.         0.83403507]
 [0.83403507 1.        ]]
Finding correlation row : 2271
Pearson's correlation score: [[1.         0.66604386]
 [0.66604386 1.        ]]
Finding correlation row : 2272
Pearson's correlation score: [[1.         0.65632582]
 [0.65632582 1.        ]]
Finding correlation row : 2273
Pearson's correlation score: [[1.         0.68638667]
 [0.68638667 1.        ]]
Finding correlation row : 2274
Pearson's correlation score: [[1.         0.81192964]
 [0.81192964 1.        ]]
Finding correlation row : 2275
Pearson's correlation score: [[1.         0.92168797]
 [0.92168797 1.        ]]
Finding correlation row : 2276
P

Pearson's correlation score: [[ 1. nan]
 [nan nan]]
Finding correlation row : 2683
Pearson's correlation score: [[1.         0.70891864]
 [0.70891864 1.        ]]
Finding correlation row : 2684
Pearson's correlation score: [[1.         0.71498416]
 [0.71498416 1.        ]]
Finding correlation row : 2685
Pearson's correlation score: [[1.         0.47703017]
 [0.47703017 1.        ]]
Finding correlation row : 2686
Pearson's correlation score: [[1.         0.78655846]
 [0.78655846 1.        ]]
Finding correlation row : 2687
Pearson's correlation score: [[1.         0.76501236]
 [0.76501236 1.        ]]
Finding correlation row : 2688
Pearson's correlation score: [[1.         0.81352717]
 [0.81352717 1.        ]]
Finding correlation row : 2689
Pearson's correlation score: [[1.         0.69391916]
 [0.69391916 1.        ]]
Finding correlation row : 2690
Pearson's correlation score: [[1.         0.84938816]
 [0.84938816 1.        ]]
Finding correlation row : 2691
Pearson's correlation score: 

Pearson's correlation score: [[1.         0.92992268]
 [0.92992268 1.        ]]
Finding correlation row : 3067
Pearson's correlation score: [[1.         0.82423463]
 [0.82423463 1.        ]]
Finding correlation row : 3068
Pearson's correlation score: [[1.         0.76960664]
 [0.76960664 1.        ]]
Finding correlation row : 3069
Pearson's correlation score: [[1.         0.80785738]
 [0.80785738 1.        ]]
Finding correlation row : 3070
Pearson's correlation score: [[1.        0.8928059]
 [0.8928059 1.       ]]
Finding correlation row : 3071
Pearson's correlation score: [[1.         0.72631031]
 [0.72631031 1.        ]]
Finding correlation row : 3072
Pearson's correlation score: [[1.         0.68146206]
 [0.68146206 1.        ]]
Finding correlation row : 3073
Pearson's correlation score: [[1.         0.86998697]
 [0.86998697 1.        ]]
Finding correlation row : 3074
Pearson's correlation score: [[1.        0.6838034]
 [0.6838034 1.       ]]
Finding correlation row : 3075
Pearson's

Pearson's correlation score: [[1.         0.85971233]
 [0.85971233 1.        ]]
Finding correlation row : 3463
Pearson's correlation score: [[1.         0.70004364]
 [0.70004364 1.        ]]
Finding correlation row : 3464
Pearson's correlation score: [[1.         0.85060417]
 [0.85060417 1.        ]]
Finding correlation row : 3465
Pearson's correlation score: [[1.         0.75434067]
 [0.75434067 1.        ]]
Finding correlation row : 3466
Pearson's correlation score: [[1.         0.98623073]
 [0.98623073 1.        ]]
Finding correlation row : 3467
Pearson's correlation score: [[1.         0.99120477]
 [0.99120477 1.        ]]
Finding correlation row : 3468
Pearson's correlation score: [[1.         0.52404774]
 [0.52404774 1.        ]]
Finding correlation row : 3469
Pearson's correlation score: [[1.         0.96092055]
 [0.96092055 1.        ]]
Finding correlation row : 3470
Pearson's correlation score: [[1.         0.94264326]
 [0.94264326 1.        ]]
Finding correlation row : 3471
P

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

